In [1]:
%matplotlib qt
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import random
import scipy
import sklearn
import sys
import datetime as dt
import math
import seaborn as sns
import os
import matplotlib.dates as mdates
from brokenaxes import brokenaxes
from matplotlib.gridspec import GridSpec

In [2]:
# initialize the DLC parameters
# initializer_coeff_precond = 0 # preconditioning parameter multipliers
# initializer_coeff_sb_su = 0 # setup/setback parameter multipliers

def my_callback_function(state):
    global ph_mag,ph_dur,sb_mag, pc1_mag, pc1_dur, su1_mag, pc2_mag, pc2_dur, su2_mag
    global low_limit, high_limit,trim_respond,hot_complaint,cold_complaint,hsp,csp
    global occupied_time
    if api.exchange.warmup_flag(state):
        return
    envNum = api.exchange.current_environment_num(state)
    if envNum == 3: # if sizing done
        tSetHtg_hndl = api.exchange.get_actuator_handle(state, "Schedule:Constant", "Schedule Value", "tSetHtg")
        tSetClg_hndl = api.exchange.get_actuator_handle(state, "Schedule:Constant", "Schedule Value", "tSetClg")
        elect_hndl = api.exchange.get_meter_handle(state,"Electricity:Facility")
        htg_hndl = api.exchange.get_meter_handle(state,"Heating:Electricity")
        clg_hndl = api.exchange.get_meter_handle(state,"Cooling:Electricity")
        tIn_office_hndl = api.exchange.get_variable_handle(state,"Zone Mean Air Temperature","Office")
        tIn_lobby_hndl = api.exchange.get_variable_handle(state,"Zone Mean Air Temperature","Lobby")

        # exchange information with EnergyPlus
        meter_reading = api.exchange.get_meter_value(state,elect_hndl)
        heating_meter_reading = api.exchange.get_meter_value(state,htg_hndl)
        cooling_meter_reading = api.exchange.get_meter_value(state,clg_hndl)
        tIn_office = api.exchange.get_variable_value(state,tIn_office_hndl)
        tIn_lobby = api.exchange.get_variable_value(state,tIn_lobby_hndl)
        month = api.exchange.month(state) 
        time_of_day = api.exchange.current_time(state)
        day_of_week = api.exchange.day_of_week(state)
        day_of_month = api.exchange.day_of_month(state)
        day_of_year = api.exchange.day_of_year(state)
        
        hours = int(math.floor(time_of_day))
        minutes = math.floor((time_of_day%1) * 60)
        if hours >23:
            hours = 23
            minutes = 60
        actTime = dt.datetime(2019,month,day_of_month,hours,0,0) + dt.timedelta(minutes=minutes)
        

        ## occupant complaint model
        cold_complaint = 0
        hot_complaint = 0 
        if (time_of_day > 8) and (time_of_day < 17) and (day_of_week > 1): # occupied periods
            occupied_time = occupied_time + 1
            if (tIn_office < low_limit-0.1) or (tIn_lobby < low_limit-0.1):
                cold_complaint = 1
            elif (tIn_office > high_limit+0.1) or (tIn_lobby > high_limit+0.1):
                hot_complaint = 1
        
        ## actuate DLC parameters
        api.exchange.set_actuator_value(state, tSetHtg_hndl, 18)
        api.exchange.set_actuator_value(state, tSetClg_hndl, 27)
        
        hsp = 18
        csp = 27

        # bank operates on weekdays and on Saturdays
        if (day_of_week > 1):
            if (time_of_day > 7 and time_of_day < 17):
                api.exchange.set_actuator_value(state, tSetHtg_hndl, 21)
                api.exchange.set_actuator_value(state, tSetClg_hndl, 24)
                hsp = 21
                csp = 24
        
        # time of use rates apply to weekdays only
        if ((month > 10) or (month < 5)) and (day_of_week > 1 and day_of_week < 7):
            if (time_of_day > 7 - ph_dur) and (time_of_day < 7): # preheat winter off-peak
                api.exchange.set_actuator_value(state, tSetHtg_hndl, 18 + ph_mag)
                hsp = 18 + ph_mag
            if (time_of_day >= 7) and (time_of_day < 11): # setback winter on-peak
                api.exchange.set_actuator_value(state, tSetHtg_hndl, 21 - sb_mag)
                hsp = 21 - sb_mag
  
        if ((month > 4) and (month < 11)) and (day_of_week > 1 and day_of_week < 7):
            if (time_of_day > 7 - pc1_dur) and (time_of_day < 7): # precool summer off-peak
                api.exchange.set_actuator_value(state, tSetClg_hndl, 27 - pc1_mag)
                csp = 27 - pc1_mag
            if (time_of_day >= 7) and (time_of_day < 11 - pc2_dur): # setup summer mid-peak
                api.exchange.set_actuator_value(state, tSetClg_hndl, 24 + su1_mag)
                csp = 24 + su1_mag
            if (time_of_day >= 11 - pc2_dur) and (time_of_day < 11): # precool summer mid-peak
                api.exchange.set_actuator_value(state, tSetClg_hndl, 24 - pc2_mag)
                csp = 24 - pc2_mag
            if (time_of_day >= 11) and (time_of_day <= 17): # setup summer on-peak
                api.exchange.set_actuator_value(state, tSetClg_hndl, 24 + su2_mag)
                csp = 24 + su2_mag

        
        # rate calculator
        cost = ((heating_meter_reading + cooling_meter_reading)/1000000*0.278)*8.7/100
        if ((month > 10) or (month < 5)) and (day_of_week > 1 and day_of_week < 7):
            if (time_of_day > 7) and (time_of_day <= 11):
                cost = ((heating_meter_reading + cooling_meter_reading)/1000000*0.278)*18.2/100
            elif (time_of_day > 11) and (time_of_day <= 17):
                cost = ((heating_meter_reading + cooling_meter_reading)/1000000*0.278)*12.2/100
            elif (time_of_day > 17) and (time_of_day < 19): 
                cost = ((heating_meter_reading + cooling_meter_reading)/1000000*0.278)*18.2/100
        
        if ((month > 4) and (month < 11)) and (day_of_week > 1 and day_of_week < 7):
            if (time_of_day > 7) and (time_of_day <= 11):
                cost = ((heating_meter_reading + cooling_meter_reading)/1000000*0.278)*12.2/100
            elif (time_of_day > 11) and (time_of_day <= 17):
                cost = ((heating_meter_reading + cooling_meter_reading)/1000000*0.278)*18.2/100
            elif (time_of_day > 17) and (time_of_day < 19): 
                cost = ((heating_meter_reading + cooling_meter_reading)/1000000*0.278)*12.2/100
        
        if trim_respond == 1:
            if ((month > 10) or (month < 5)) and (day_of_week > 1 and day_of_week < 7):
                if (cold_complaint == 0): # no cold complaint
                    # DLC is unnecessarily aggressive
                    if ((tIn_office - (21 - sb_mag)) >= 0.5) and ((tIn_lobby - (21 - sb_mag)) >= 0.5) and (time_of_day > 10.9 and time_of_day < 11.1): # postmature and no complaint
                        ph_dur = min(max(ph_dur - 0.1,0),3)
                        ph_mag = min(max(ph_mag - 0.1,0),6)
                        sb_mag = min(max(sb_mag - 0.1,0),3)
                    # DLC is not aggressive enough
                    elif (((tIn_office - (21 - sb_mag)) < 0.2) or ((tIn_lobby - (21 - sb_mag)) < 0.2)) and (time_of_day > 10.9 and time_of_day < 11.1): # premature and no complaint
                        ph_dur = min(max(ph_dur + 0.1,0),3)
                        ph_mag = min(max(ph_mag + 0.1,0),6)
                        sb_mag = min(max(sb_mag + 0.1,0),3)
                else: # cold complaint
                    ph_dur = min(max(ph_dur + 0.1,0),3)
                    ph_mag = min(max(ph_mag + 0.1,0),6)
                    sb_mag = min(max(sb_mag - 0.2,0),3)

            if ((month > 4) and (month < 11)) and (day_of_week > 1 and day_of_week < 7):
                if (hot_complaint == 0): # no hot complaint
                    # DLC is unnecessarily aggressive
                    if ((tIn_office - (24 + su1_mag)) <= -0.5) and ((tIn_lobby - (24 + su1_mag)) <= -0.5) and (time_of_day > 10.9 - pc2_dur and time_of_day < 11.1 - pc2_dur): # postmature and no complaint
                        pc1_dur = min(max(pc1_dur - 0.1,0),3)
                        pc1_mag = min(max(pc1_mag - 0.1,0),6)
                        su1_mag = min(max(su1_mag - 0.1,0),3)
                    # DLC is not aggressive enough
                    elif (((tIn_office - (24 + su1_mag)) > -0.2) or ((tIn_lobby - (24 + su1_mag)) > -0.2)) and (time_of_day > 10.9 - pc2_dur and time_of_day < 11.1 - pc2_dur): # premature and no complaint
                        pc1_dur = min(max(pc1_dur + 0.1,0),3)
                        pc1_mag = min(max(pc1_mag + 0.1,0),6)
                        su1_mag = min(max(su1_mag + 0.1,0),3)
                else: # hot complaint
                    pc1_dur = min(max(pc1_dur + 0.1,0),3)
                    pc1_mag = min(max(pc1_mag + 0.1,0),6)
                    su1_mag = min(max(su1_mag - 0.2,0),3)                

            if ((month > 4) and (month < 11)) and (day_of_week > 1 and day_of_week < 7):
                if (hot_complaint == 0) and (cold_complaint == 0): # no hot complaint
                    if ((tIn_office - (24 + su2_mag)) <= -0.5) and ((tIn_lobby - (24 + su2_mag)) <= -0.5) and (time_of_day > 16.9 and time_of_day < 17.1): # postmature and no complaint
                        pc2_dur = min(max(pc2_dur - 0.1,0),3)
                        pc2_mag = min(max(pc2_mag - 0.1,0),3)
                        su2_mag = min(max(su2_mag - 0.1,0),3)
                    elif (((tIn_office - (24 + su2_mag)) > -0.2) or ((tIn_lobby - (24 + su2_mag)) > -0.2)) and (time_of_day > 16.9 and time_of_day < 17.1): # premature and no complaint
                        pc2_dur = min(max(pc2_dur + 0.1,0),3)
                        pc2_mag = min(max(pc2_mag + 0.1,0),3)
                        su2_mag = min(max(su2_mag + 0.1,0),3)             
                elif (cold_complaint == 1): # reduce precooling
                    pc2_dur = min(max(pc2_dur - 0.1,0),3)
                    pc2_mag = min(max(pc2_mag - 0.2,0),3)
                elif (hot_complaint == 1): # increase precooling and reduce setup
                    pc2_dur = min(max(pc2_dur + 0.1,0),3)
                    pc2_mag = min(max(pc2_mag + 0.1,0),3)
                    su2_mag = min(max(su2_mag - 0.2,0),3)
        
        x = [ph_dur,ph_mag,sb_mag,pc1_dur,pc1_mag,su1_mag,pc2_dur,pc2_mag,su2_mag]
        diagnostic.append([ph_dur,ph_mag,sb_mag,pc1_dur,pc1_mag,su1_mag,pc2_dur,pc2_mag,su2_mag, cost, tIn_office,tIn_lobby,hsp,csp,time_of_day,day_of_week,hot_complaint,cold_complaint])
        

In [6]:
# total occupied time steps = 10955
sys.path.insert(0, 'C:\EnergyPlusV23-1-0')  # add E-Plus directory to path to be able to import API
from pyenergyplus.api import EnergyPlusAPI #import EnergyPlus library
api = EnergyPlusAPI()

scenarios_info = pd.read_excel('scenarios_def_TR.xlsx')
active_cases = scenarios_info[scenarios_info.active == 1]

for i, row in active_cases.iterrows():
    diagnostic = []
    idf_file = "Models_heatpump/"+row.idf_file
    epw_file = row.epw_file
    output_dir = "Results_TR/Scenario_{}".format(row.ID)
    offset_x = eval(row.offset_x)
    offset_y = eval(row.offset_y)
    
    ph_mag = offset_x[0]
    ph_dur = offset_x[1]
    sb_mag = offset_x[2]

    pc1_mag = offset_y[0]
    pc1_dur = offset_y[1]
    su1_mag = offset_y[2]

    pc2_mag = offset_y[3]
    pc2_dur = offset_y[4]
    su2_mag = offset_y[5]

    trim_respond = row.TR # DLC parameter updates if set to one, else no update
    
    [low_limit, high_limit] = eval(row.comfort_range)
    occupied_time = 0
    
    state = api.state_manager.new_state() 
    api.runtime.callback_begin_system_timestep_before_predictor(state,my_callback_function)
    # -x short form to run expandobjects for HVACtemplates. see EnergyPlusEssentials.pdf p16
    cmd_args = ['-w',epw_file, '-d', output_dir,'-x',idf_file]
    api.runtime.run_energyplus(state,cmd_args)
    api.state_manager.delete_state(state)
    
    column_names=['ph_dur','ph_mag','sb_mag','pc1_dur','pc1_mag','su1_mag','pc2_dur','pc2_mag','su2_mag',\
             'cost','tIn_office','tIn_lobby','hsp','csp','time','day','hot_complaint','cold_complaint']
    df = pd.DataFrame(data = diagnostic, columns=column_names)
    df.to_csv(output_dir+"/diagnostic.csv")

print(occupied_time)    

10955


In [ ]:
df_temp = pd.DataFrame([time_list,electricity]).transpose()
df_temp.to_csv("electricity_cost_array_TR.csv")

In [7]:
df = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(9))
df.cost.sum()

2125.3813705740795

# Temperature profiles and complaints

In [90]:
case_no = 24
time_range = pd.date_range(start='2019-01-01 00:15:00', end='2020-1-1 00:00:00', freq='15T')
df = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
df["timestamp"] = time_range
plt.plot(df['su2_mag'])
print(sum(df["cost"]))
print(sum(df['hot_complaint'])+sum(df['cold_complaint']))

2896.162322136576
60


In [91]:
fig, ax1 = plt.subplots(figsize=(8,5))
ax2 = ax1.twinx()
ax1.plot(df['timestamp'],df['hsp'], label='Heating SP',color="orangered",linestyle=":")
ax1.plot(df['timestamp'],df['csp'], label='Cooling SP',color="navy",linestyle=":")
ax1.plot(df['timestamp'],df['tIn_office'],'k', label='Office temp')
ax1.plot(df['timestamp'],df['tIn_lobby'],'k--', label='Lobby temp')
ax2.plot(df['timestamp'],df["cost"],'r', label='Cost')
ax2.scatter(df['timestamp'],df['cold_complaint'],s =20, c = 'darkgray', label='Complaints')
month = 2
day = 18  #21
ax1.set_xlim([dt.datetime(2019, month, day, 3, 0, 0), dt.datetime(2019, month, day+4, 20, 0, 0)]) 
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax1.set_xlabel('Time of day (h)')
ax1.set_ylabel('Temperature ($^{\circ}C$)', color='k')
ax2.set_ylabel('Electricity cost ($/15-min)', color='r');
# legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, ncol=3, loc="upper center")
ax1.set_title("February 18$^{th}$-22$^{th}$")
# plt.savefig("Feb_profile.png", dpi=300)

In [92]:
fig, ax1 = plt.subplots( figsize=(8,5))
ax2 = ax1.twinx()
ax1.plot(df['timestamp'],df['hsp'], label='Heating SP',color="orangered",linestyle=":")
ax1.plot(df['timestamp'],df['csp'],'k:', label='Cooling SP',color="navy",linestyle=":")
ax1.plot(df['timestamp'],df['tIn_office'],'k', label='Office temp.')
ax1.plot(df['timestamp'],df['tIn_lobby'],'k--', label='Lobby temp.')
ax2.plot(df['timestamp'],df["cost"],'r', label='Cost')
ax2.scatter(df['timestamp'],df['cold_complaint'],s =20, c = 'darkgray', label='Complaints')
month = 5
day = 13
ax1.set_xlim([dt.datetime(2019, month, day, 4, 0, 0), dt.datetime(2019, month, day+3, 20, 0, 0)])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax1.set_xlabel('Time of day (h)')
ax1.set_ylabel('Temperature ($^{\circ}C$)', color='k')
ax2.set_ylabel('Electricity cost ($/15-min)', color='r');
# legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, ncol=3, loc="upper center")
ax1.set_title("May 13$^{th}$-16$^{th}$")
# plt.savefig("May_profile.png", dpi=300)

C:\Users\Hussein Elehwany\AppData\Local\Temp\ipykernel_25264\3116233913.py:4: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "k:" (-> linestyle=':'). The keyword argument will take precedence.
  ax1.plot(df['timestamp'],df['csp'],'k:', label='Cooling SP',color="navy",linestyle=":")
C:\Users\Hussein Elehwany\AppData\Local\Temp\ipykernel_25264\3116233913.py:4: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k:" (-> color='k'). The keyword argument will take precedence.
  ax1.plot(df['timestamp'],df['csp'],'k:', label='Cooling SP',color="navy",linestyle=":")


In [93]:
month = [1 ,2]
day = [11, 19]
date_labels = ["January 11$^{th}$", "February 19$^{th}$"]
fig, ax = plt.subplots(2,1, figsize=(8,5))
for i in range(2):
    ax2 = ax[i].twinx()
    ax[i].plot(df['timestamp'],df['hsp'],'k:', label='Heating SP',color="orangered",linestyle=":")
    ax[i].plot(df['timestamp'],df['csp'],'k:', label='Cooling SP',color="navy",linestyle=":")
    ax[i].plot(df['timestamp'],df['tIn_office'],'k', label='Office temp')
    ax[i].plot(df['timestamp'],df['tIn_lobby'],'k--', label='Lobby temp')
    ax2.plot(df['timestamp'],df["cost"],'r', label='Cost')
    ax2.scatter(df['timestamp'],df['cold_complaint'],s =20, c = 'darkgray', label='Complaints')

    ax[i].set_xlim([dt.datetime(2019, month[i], day[i], 3, 0, 0), dt.datetime(2019, month[i], day[i], 20, 0, 0)]) 
    ax[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    
    ax[i].set_ylabel('Temperature ($^{\circ}C$)', color='k')
    ax2.set_ylabel('Electricity cost ($/15-min)', color='r');
    # legend
    lines, labels = ax[i].get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax[i].text(0.5, 0.9, date_labels[i],horizontalalignment='center', transform=ax[i].transAxes)
    
    
#     ax[i].set_title("February 18$^{th}$-22$^{th}$")
ax[1].set_xlabel('Time of day (h)')   
ax[0].legend(lines + lines2, labels + labels2, ncol=3, bbox_to_anchor=(0.5, 1.02), loc='lower center', borderaxespad=0)

# plt.savefig("Jan_Feb_profile.png", dpi=300)
                   

C:\Users\Hussein Elehwany\AppData\Local\Temp\ipykernel_25264\4195013878.py:7: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "k:" (-> linestyle=':'). The keyword argument will take precedence.
  ax[i].plot(df['timestamp'],df['hsp'],'k:', label='Heating SP',color="orangered",linestyle=":")
C:\Users\Hussein Elehwany\AppData\Local\Temp\ipykernel_25264\4195013878.py:7: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k:" (-> color='k'). The keyword argument will take precedence.
  ax[i].plot(df['timestamp'],df['hsp'],'k:', label='Heating SP',color="orangered",linestyle=":")
C:\Users\Hussein Elehwany\AppData\Local\Temp\ipykernel_25264\4195013878.py:8: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "k:" (-> linestyle=':'). The keyword argument will take precedence.
  ax[i].plot(df['timestamp'],df['csp'],'k:', label='Cooling SP',color="navy",

# DLC scatter 

In [ ]:
scenarios_info = pd.read_excel('scenarios_def_TR.xlsx')
fig, ax = plt.subplots(2, 4, figsize=(15,10))
cases = [r'[0,0,0]', r'[2,1,1]', r'[4,2,2]', r'[6,3,3]',r'[0,0,1]',r'[0,0,2]',r'[0,0,3]']
labels = ["no DLC", "DLC1", "DLC2", "DLC3", "DLC4", "DLC5", "DLC6", "TR"]
for j in range(8):
    if j < 7:
        TR_cases = scenarios_info.loc[(scenarios_info.TR == 0) & (scenarios_info.offset_x == cases[j])]
    else:
        TR_cases = scenarios_info.loc[(scenarios_info.TR == 1) ]
    TR_cases.head()
    scenarios = TR_cases.ID.values
    scenarios
    elect_cost = []
    complaints = []
    for i in scenarios:
        df = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(i))

        elect_cost.append(sum(df["cost"]))
        complaints.append(sum(df['hot_complaint'])+sum(df['cold_complaint']))
    df_plot = pd.DataFrame([elect_cost, complaints]).transpose()
    df_plot.columns=["cost","complaints"]

    if j < 4:
        sns.scatterplot(data=df_plot,x="cost", y="complaints", ax=ax[0][j])\
        .set(ylim=[0,6100],xlim=[900,3500],xlabel=None, title=labels[j])
        
    else:
        sns.scatterplot(data=df_plot,x="cost", y="complaints", ax=ax[1][j-4])\
        .set(ylim=[0,6100],xlim=[900,3500], title=labels[j]) 

# Evolution of actions

In [ ]:
case_no = 24
time_range = pd.date_range(start='2019-01-01 00:15:00', end='2020-1-1 00:00:00', freq='15T')
df = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
df["timestamp"] = time_range
df.info()
wntr_lims_1 = (dt.datetime(2019, 1, 1), dt.datetime(2019, 5, 1))
wntr_lims_2 = (dt.datetime(2019, 11, 1), dt.datetime(2020, 1, 1))
bax = brokenaxes(xlims=(wntr_lims_1, wntr_lims_2),hspace=.05)

bax.plot(df["timestamp"], df[['ph_dur','ph_mag','sb_mag']])

In [48]:
fig, ax = plt.subplots(3,1, figsize=(6,5))
case_no = 8
time_range = pd.date_range(start='2019-01-01 00:15:00', end='2020-1-1 00:00:00', freq='15T')
df1 = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
df1["timestamp"] = time_range
case_no = 16
df2 = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
df2["timestamp"] = time_range
case_no = 24
df3 = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
df3["timestamp"] = time_range

ax[0].plot(df1["timestamp"], df1['ph_dur'], df2["timestamp"], df2['ph_dur'], df3["timestamp"], df3['ph_dur'])
ax[1].plot(df1["timestamp"], df1['ph_mag'], df2["timestamp"], df2['ph_mag'], df3["timestamp"], df3['ph_mag'])
ax[2].plot(df1["timestamp"], df1['sb_mag'], df2["timestamp"], df2['sb_mag'], df3["timestamp"], df3['sb_mag'])

for i in range(3):
    ax[i].xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
#     ax[i].set_xticklabels(["Jan", "Mar", "May", "Jul", "Sep", "Nov", "Jan"])

ax[0].set(ylabel = "$\Delta t_{ph}\ [h]$", ylim=[0,3.1])
ax[1].set(ylabel= "$\Delta T_{ph}\ [^{\circ}C]$", ylim=[0,6.1])
ax[2].set(ylabel= "$\Delta T_{sb}\ [^{\circ}C]$", ylim=[0,3.1])
ax[0].set_title("Normal-sized - occupant preferences [20$^{\circ}C$ -26$^{\circ}C$]")
ax[2].legend(["High env.","Average env.", "Poor env."], bbox_to_anchor=(0.5, -0.35), loc='center', borderaxespad=0, ncol=3)
# plt.savefig("Different_envelopes_normal_size_1.png", dpi=300)

In [49]:
fig, ax = plt.subplots(3,1, figsize=(6,5))
case_no = 16
time_range = pd.date_range(start='2019-01-01 00:15:00', end='2020-1-1 00:00:00', freq='15T')
df = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
df["timestamp"] = time_range
ax[0].plot(df["timestamp"], df['ph_dur'])
ax[1].plot(df["timestamp"], df['ph_mag'])
ax[2].plot(df["timestamp"], df['sb_mag'])



case_no = 40
df = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
df["timestamp"] = time_range
ax[0].plot(df["timestamp"], df[['ph_dur']])
ax[1].plot(df["timestamp"], df['ph_mag'])
ax[2].plot(df["timestamp"], df['sb_mag'])

case_no = 64
df = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
df["timestamp"] = time_range
ax[0].plot(df["timestamp"], df[['ph_dur']])
ax[1].plot(df["timestamp"], df['ph_mag'])
ax[2].plot(df["timestamp"], df['sb_mag'])

for i in range(3):
    ax[i].xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
    ax[i].set_xticklabels(["Jan", "Mar", "May", "Jul", "Sep", "Nov", "Jan"])

ax[0].set(ylabel = "$\Delta t_{ph}\ [h]$", ylim=[0,3.1])
ax[1].set(ylabel= "$\Delta T_{ph}\ [^{\circ}C]$", ylim=[0,6.1])
ax[2].set(ylabel= "$\Delta T_{sb}\ [^{\circ}C]$", ylim=[0,3.1])
ax[0].set_title("Average envelope - occupant preferences [20$^{\circ}C$ -26$^{\circ}C$]")
ax[2].legend(["Normal","Undersized", "Oversized"], bbox_to_anchor=(0.5, -0.35), loc='center', borderaxespad=0, ncol=3)
# plt.savefig("Different_sizes_normal_env_1.png", dpi=300)

C:\Users\Hussein Elehwany\AppData\Local\Temp\ipykernel_25264\3446489926.py:28: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[i].set_xticklabels(["Jan", "Mar", "May", "Jul", "Sep", "Nov", "Jan"])


In [50]:
fig, ax = plt.subplots(3,2, figsize=(7,6))
case_no = 8
time_range = pd.date_range(start='2019-01-01 00:15:00', end='2020-1-1 00:00:00', freq='15T')
df1 = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
df1["timestamp"] = time_range
case_no = 16
df2 = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
df2["timestamp"] = time_range
case_no = 24
df3 = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
df3["timestamp"] = time_range

ax[0,0].plot(df1["timestamp"], df1['pc1_dur'], df2["timestamp"], df2['pc1_dur'], df3["timestamp"], df3['pc1_dur'])
ax[1,0].plot(df1["timestamp"], df1['pc1_mag'], df2["timestamp"], df2['pc1_mag'], df3["timestamp"], df3['pc1_mag'])
ax[2,0].plot(df1["timestamp"], df1['su1_mag'], df2["timestamp"], df2['su1_mag'], df3["timestamp"], df3['su1_mag'])

ax[0,1].plot(df1["timestamp"], df1['pc2_dur'], df2["timestamp"], df2['pc2_dur'], df3["timestamp"], df3['pc2_dur'])
ax[1,1].plot(df1["timestamp"], df1['pc2_mag'], df2["timestamp"], df2['pc2_mag'], df3["timestamp"], df3['pc2_mag'])
ax[2,1].plot(df1["timestamp"], df1['su2_mag'], df2["timestamp"], df2['su2_mag'], df3["timestamp"], df3['su2_mag'])

for i in range(3):
    for j in range(2):
        ax[i,j].xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
        ax[i,j].set_xlim(dt.datetime(2019,5,1), dt.datetime(2019,11,1))
        ax[i,j].set_xticklabels(["May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov"])

ax[0,0].set(ylabel="$\Delta t_{pc1}\ [h]$", ylim = [0,3.1])
ax[1,0].set(ylabel="$\Delta T_{pc1}\ [^{\circ}C]$", ylim = [0,6.1])
ax[2,0].set(ylabel="$\Delta T_{su1}\ [^{\circ}C]$", ylim = [0,3.1])
ax[0,1].set(ylabel="$\Delta t_{pc2}\ [h]$", ylim = [0,3.1])
ax[1,1].set(ylabel="$\Delta T_{pc2}\ [^{\circ}C]$", ylim = [0,3.1])
ax[2,1].set(ylabel="$\Delta T_{su2}\ [^{\circ}C]$", ylim = [0,3.1])
# ax[0,0].set_title("Normal-sized - occupant preferences [20$^{\circ}C$ -26$^{\circ}C$]")
fig.suptitle("Normal-sized - occupant preferences [20$^{\circ}C$ -26$^{\circ}C$]", x=0.5, y=0.92)
ax[2,0].legend(["High env.","Average env.", "Poor env."], bbox_to_anchor=(0.3, -0.3), loc='center left', borderaxespad=0, ncol=3)
# plt.savefig("Different_envelopes_normal_size_summer.png", dpi=300)

C:\Users\Hussein Elehwany\AppData\Local\Temp\ipykernel_25264\4082352618.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[i,j].set_xticklabels(["May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov"])


In [51]:
fig, ax = plt.subplots(3,2, figsize=(7,6))
case_no = 16
time_range = pd.date_range(start='2019-01-01 00:15:00', end='2020-1-1 00:00:00', freq='15T')
df1 = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
df1["timestamp"] = time_range
case_no = 40
df2 = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
df2["timestamp"] = time_range
case_no = 64
df3 = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
df3["timestamp"] = time_range

ax[0,0].plot(df1["timestamp"], df1['pc1_dur'], df2["timestamp"], df2['pc1_dur'], df3["timestamp"], df3['pc1_dur'])
ax[1,0].plot(df1["timestamp"], df1['pc1_mag'], df2["timestamp"], df2['pc1_mag'], df3["timestamp"], df3['pc1_mag'])
ax[2,0].plot(df1["timestamp"], df1['su1_mag'], df2["timestamp"], df2['su1_mag'], df3["timestamp"], df3['su1_mag'])

ax[0,1].plot(df1["timestamp"], df1['pc2_dur'], df2["timestamp"], df2['pc2_dur'], df3["timestamp"], df3['pc2_dur'])
ax[1,1].plot(df1["timestamp"], df1['pc2_mag'], df2["timestamp"], df2['pc2_mag'], df3["timestamp"], df3['pc2_mag'])
ax[2,1].plot(df1["timestamp"], df1['su2_mag'], df2["timestamp"], df2['su2_mag'], df3["timestamp"], df3['su2_mag'])

for i in range(3):
    for j in range(2):
        ax[i,j].xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
        ax[i,j].set_xlim(dt.datetime(2019,5,1), dt.datetime(2019,11,1))
        ax[i,j].set_xticklabels(["May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov"])

ax[0,0].set(ylabel="$\Delta t_{pc1}\ [h]$", ylim = [0,3.1])
ax[1,0].set(ylabel="$\Delta T_{pc1}\ [^{\circ}C]$", ylim = [0,6.1])
ax[2,0].set(ylabel="$\Delta T_{su1}\ [^{\circ}C]$", ylim = [0,3.1])
ax[0,1].set(ylabel="$\Delta t_{pc2}\ [h]$", ylim = [0,3.1])
ax[1,1].set(ylabel="$\Delta T_{pc2}\ [^{\circ}C]$", ylim = [0,3.1])
ax[2,1].set(ylabel="$\Delta T_{su2}\ [^{\circ}C]$", ylim = [0,3.1])
# ax[0,0].set_title("Normal-sized - occupant preferences [20$^{\circ}C$ -26$^{\circ}C$]")
fig.suptitle("Normal-sized - occupant preferences [20$^{\circ}C$ -26$^{\circ}C$]", x=0.5, y=0.92)
ax[2,0].legend(["good env.","normal env.", "poor env."], bbox_to_anchor=(0.3, -0.3), loc='center left', borderaxespad=0, ncol=3)
# plt.savefig("Different_sizes_normal_env_summer.png", dpi=300)

C:\Users\Hussein Elehwany\AppData\Local\Temp\ipykernel_25264\2061211175.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[i,j].set_xticklabels(["May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov"])


# Save all results

In [ ]:
scenarios_info = pd.read_excel('scenarios_def_TR.xlsx')
scenarios = scenarios_info.ID.values
container = []
for i in scenarios:
        df = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(i))
        container.append([i, sum(df["cost"]), sum(df['hot_complaint'])+sum(df['cold_complaint'])])
df = pd.DataFrame(container, columns=["ID", "cost","complaints"])
df.to_csv("all_results.csv")
print("done")

# scenarios boxplots 
boxes for scenarios, each box has 6 DLCs, 1 point for no dlc and one for TR

In [5]:
df = pd.read_csv("all_results.csv")
df.head()

container_base = []
container_dlc1 = []
container_dlc2 = []
container_dlc3 = []
container_dlc4 = []
container_dlc5 = []
container_dlc6 = []
container_TR = []
counter = 0
for i in range(288):
    idx = i%8
    if idx == 0:   
        container_base.append(df["cost"][i])
    elif idx == 1:
        container_dlc1.append(df["cost"][i])
    elif idx == 2:
        container_dlc2.append(df["cost"][i])
    elif idx == 3:
        container_dlc3.append(df["cost"][i])
    elif idx == 4:
        container_dlc4.append(df["cost"][i])
    elif idx == 5:
        container_dlc5.append(df["cost"][i])
    elif idx == 6:
        container_dlc6.append(df["cost"][i])
    elif idx == 7:   
        container_TR.append(df["cost"][i])

# x_labels = ["ge/ns/1","ne/ns/1","pe/ns/1","ge/us/1","ne/us/1","pe/us/1","ge/os/1","ne/os/1","pe/os/1",
#             "ge/ns/2","ne/ns/2","pe/ns/2","ge/us/2","ne/us/2","pe/us/2","ge/os/2","ne/os/2","pe/os/2",
#             "ge/ns/3","ne/ns/3","pe/ns/3","ge/us/3","ne/us/3","pe/us/3","ge/os/3","ne/os/3","pe/os/3",
#             "ge/ns/4","ne/ns/4","pe/ns/4","ge/us/4","ne/us/4","pe/us/4","ge/os/4","ne/os/4","pe/os/4"]
x_labels = ["he/ns/1","ae/ns/1","pe/ns/1","he/us/1","ae/us/1","pe/us/1","he/os/1","ae/os/1","pe/os/1",
            "he/ns/2","ae/ns/2","pe/ns/2","he/us/2","ae/us/2","pe/us/2","he/os/2","ae/os/2","pe/os/2",
            "he/ns/3","ae/ns/3","pe/ns/3","he/us/3","ae/us/3","pe/us/3","he/os/3","ae/os/3","pe/os/3",
            "he/ns/4","ae/ns/4","pe/ns/4","he/us/4","ae/us/4","pe/us/4","he/os/4","ae/os/4","pe/os/4"]
a = np.arange(0,36)
df_dlc = pd.DataFrame([container_dlc1,container_dlc2,container_dlc3,container_dlc4,container_dlc5,container_dlc6])
fig, ax = plt.subplots(figsize=(12,8))
sns.scatterplot(x = a, y = container_base)
sns.scatterplot(x = a, y = container_TR)
sns.boxplot(df_dlc, color="lightcoral").set(xticklabels=x_labels)
plt.xticks(rotation = 60)
plt.ylabel("Annual cost [$]", fontsize=12)
plt.xlabel("Scenarios - envelopes, HVAC sizes, occupant preferences", fontsize=12)
ax.legend(["No DLC","T&R","baseline DLCs"], loc="upper right", fontsize=12)

plt.savefig("boxplots_cost_new_xticks.png", dpi=300)

In [6]:
total_occ = 10955
# x_labels = ["ge/ns/1","ne/ns/1","pe/ns/1","ge/us/1","ne/us/1","pe/us/1","ge/os/1","ne/os/1","pe/os/1",
#             "ge/ns/2","ne/ns/2","pe/ns/2","ge/us/2","ne/us/2","pe/us/2","ge/os/2","ne/os/2","pe/os/2",
#             "ge/ns/3","ne/ns/3","pe/ns/3","ge/us/3","ne/us/3","pe/us/3","ge/os/3","ne/os/3","pe/os/3",
#             "ge/ns/4","ne/ns/4","pe/ns/4","ge/us/4","ne/us/4","pe/us/4","ge/os/4","ne/os/4","pe/os/4"]
x_labels = ["he/ns/1","ae/ns/1","pe/ns/1","he/us/1","ae/us/1","pe/us/1","he/os/1","ae/os/1","pe/os/1",
            "he/ns/2","ae/ns/2","pe/ns/2","he/us/2","ae/us/2","pe/us/2","he/os/2","ae/os/2","pe/os/2",
            "he/ns/3","ae/ns/3","pe/ns/3","he/us/3","ae/us/3","pe/us/3","he/os/3","ae/os/3","pe/os/3",
            "he/ns/4","ae/ns/4","pe/ns/4","he/us/4","ae/us/4","pe/us/4","he/os/4","ae/os/4","pe/os/4"]
df = pd.read_csv("all_results.csv")
df.head()

container_base = []
container_dlc1 = []
container_dlc2 = []
container_dlc3 = []
container_dlc4 = []
container_dlc5 = []
container_dlc6 = []
container_TR = []
counter = 0
for i in range(288):
    idx = i%8
    if idx == 0:   
        container_base.append(df["complaints"][i]/total_occ)
    elif idx == 1:
        container_dlc1.append(df["complaints"][i]/total_occ)
    elif idx == 2:
        container_dlc2.append(df["complaints"][i]/total_occ)
    elif idx == 3:
        container_dlc3.append(df["complaints"][i]/total_occ)
    elif idx == 4:
        container_dlc4.append(df["complaints"][i]/total_occ)
    elif idx == 5:
        container_dlc5.append(df["complaints"][i]/total_occ)
    elif idx == 6:
        container_dlc6.append(df["complaints"][i]/total_occ)
    elif idx == 7:   
        container_TR.append(df["complaints"][i]/total_occ)

a = np.arange(0,36)
# plt.scatter(a, container_base)
# plt.scatter(a, container_TR)
df_dlc = pd.DataFrame([container_dlc1,container_dlc2,container_dlc3,container_dlc4,container_dlc5,container_dlc6])

fig, ax = plt.subplots(figsize=(12,8))

sns.scatterplot(x = a, y = container_base) #, label="no DLC"
sns.scatterplot(x = a, y = container_TR)  #, label= "T&R"
sns.boxplot(df_dlc, color="lightcoral").set(xticklabels=x_labels)
plt.xticks(rotation = 60)
plt.ylabel("$f_{discomfort}$", fontsize=15)
plt.xlabel("Scenarios - envelopes, HVAC sizes, occupant preferences", fontsize=12)
ax.legend(["No DLC","T&R","baseline DLCs"], loc="upper right", fontsize=12)
 
plt.savefig("boxplots_complaints_new_xticks.png", dpi=300)

# Compare models

In [ ]:
scenarios_info = pd.read_excel('scenarios_def_TR.xlsx')
TR_cases = scenarios_info[scenarios_info.TR == 1]
scenarios = TR_cases["ID"].values
time_range = pd.date_range(start='2019-01-01 00:15:00', end='2020-1-1 00:00:00', freq='15T')
elect_cost = []
complaints = []
for i in scenarios:
    df = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(i))
    df["timestamp"] = time_range
    elect_cost.append(sum(df["cost"]))
    complaints.append(sum(df['hot_complaint'])+sum(df['cold_complaint']))
#     complaints.append(sum(df['cold_complaint']))
    
df_plot = pd.DataFrame([elect_cost, complaints]).transpose()
df_plot.columns=["cost","complaints"]
fig, ax = plt.subplots(2, 1, figsize=(25,10))
sns.barplot(data=df_plot,x=scenarios, y="cost", ax=ax[0])  
sns.barplot(data=df_plot,x=scenarios, y="complaints", ax=ax[1])  

In [ ]:
# dont use again
df = pd.read_excel("temp_index.xlsx")
df.info()
# for i in range(288):
    ## os.rename("Results_TR/Scenario_{}".format(df.old[i]), "Results_TR/Scenario_{}".format(df.new[i]))


In [ ]:
case_no = 1
df = pd.read_csv("Results_TR/Scenario_{}/diagnostic.csv".format(case_no))
print(sum(df["cost"]))
print(sum(df['hot_complaint'])+sum(df['cold_complaint']))